<a href="https://colab.research.google.com/github/harrymkwn/InfluenceAnalysis/blob/master/AllCrowdFlower_App_Ek_cF_clean_our_combined.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import  drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
from gensim.models import Word2Vec
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from gensim.models import KeyedVectors
from sklearn.model_selection import train_test_split

import nltk
from nltk.corpus import stopwords
nltk.download('all')
device_name = tf.test.gpu_device_name()
print(device_name)
df = pd.read_csv('/content/drive/MyDrive/InfluenceAnalysis/InfluenceAnalysis/crowdFlower/cF_clean_our_combined.csv')
df = df.sample(frac=1,random_state=32)

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package biocreative_ppi is already up-to-date!
[nltk_data]    | Downloading package brown to /root/nltk_data...
[nltk_data]    |   Package brown is already up-to-date!
[nltk_data]    | Downloading package brown_tei to /root/nltk_data...
[nltk_data]    |   Package brown_tei is already up-to-date!
[nltk_data]    | Downloading package cess_cat to /root/nltk_data...
[nltk_data]    |   Package cess_cat is already up-to-date!
[nltk_data]    | Downloading package cess_esp to /root/nltk_data...
[nltk_data]    |   Package cess_esp is already up-to-date!
[nltk_data]    | Downloading packag

In [3]:
df.head()

,emotion,tweets
35975,1,Praying for love in a lap dance and paying in...
17590,2,trying to fix my internet connection gues...
38071,0,starting an account here on twitter
23304,2,hmmm wrong link ignore my tweet
26990,0,thanks before the major chop


In [7]:
print(df.emotion.unique())
no_labels=4

[1 2 0 3]


In [8]:
category_dict = {}
for i in range(no_labels):
  category_dict[i]=[0 for j in range(no_labels)]
  category_dict[i][i]=1
data_tweet = [str(x).lower().split() for x in df['tweets']]
data_cat = np.array([np.array(category_dict[x]) for x in df['emotion']])
print(data_cat[:5])

[[0 1 0 0]
 [0 0 1 0]
 [1 0 0 0]
 [0 0 1 0]
 [1 0 0 0]]


In [9]:
print(data_tweet[:5])
print(len(data_tweet))

[['praying', 'for', 'love', 'in', 'a', 'lap', 'dance', 'and', 'paying', 'in', 'naivety', 'less', 'than', 'one', 'of', 'my', 'fav', 'songs'], ['trying', 'to', 'fix', 'my', 'internet', 'connection', 'guess', 'my', 'prayers', 'have', 'been', 'answered', 'and', 'i', 'wont', 'have', 'any', 'study', 'distractions', 'ugh'], ['starting', 'an', 'account', 'here', 'on', 'twitter'], ['hmmm', 'wrong', 'link', 'ignore', 'my', 'tweet'], ['thanks', 'before', 'the', 'major', 'chop']]
40000


In [10]:
# Parameters
Min_count = 0
Embedding_size = 200
Window_size = 5
Negative_sampling = 00

In [11]:
w2v_sg = Word2Vec(min_count=Min_count,
                     window=Window_size,
                     size=Embedding_size,
                     negative=Negative_sampling,sg=1)
w2v_sg.build_vocab(data_tweet)
w2v_sg.train(data_tweet, total_examples=w2v_sg.corpus_count, epochs=5)
w2v_cbow = Word2Vec(min_count=Min_count,
                     window=Window_size,
                     size=Embedding_size,
                     negative=Negative_sampling,sg=0)
w2v_cbow.build_vocab(data_tweet)
w2v_cbow.train(data_tweet, total_examples=w2v_cbow.corpus_count, epochs=5)
w2v_sg.wv.init_sims(True)
w2v_cbow.wv.init_sims(True)

In [12]:
vocab_sg = w2v_sg.wv.vocab
vocab_sg = [x for x in vocab_sg]
vocab_cbow = w2v_cbow.wv.vocab
vocab_cbow = [x for x in vocab_cbow] 
vocab_glove = {}
with open("/content/drive/My Drive/InfluenceAnalysis/InfluenceAnalysis/glove/glove.twitter.27B.200d.txt", 'r', encoding="utf-8") as f:
  for line in f:
      values = line.split()
      word = values[0]
      vector = np.asarray(values[1:], "float32")
      vocab_glove[word] = vector

In [13]:
batch_size = 1000
Max_input_size = max([len(x) for x in data_tweet])
print(Max_input_size)

46


In [15]:
vocab_nrc={}
position={'anger':0,'anticipation':1,'disgust':2,'fear':3,'joy':4,'negative':5,'positive':6,'sadness':7,'surprise':8,'trust':9}
with open("/content/drive/MyDrive/InfluenceAnalysis/InfluenceAnalysis/Data/NRC/NRC-Emotion-Lexicon-Wordlevel-v0.92.txt", 'r', encoding="utf-8") as f:
  nrc=[]
  for line in f:
      nrc+=[line]
for x in range(len(nrc)):
  nrc[x]=nrc[x].replace('--',' ')
  nrc[x]=nrc[x].replace(', ',' ')
  nrc[x]=nrc[x].replace('\t',' ')
  nrc[x]=nrc[x].replace('\n','')
  nrc[x]=nrc[x].split(' ')
for x in nrc:
  if len(x)==3:
    vocab_nrc[x[0]]=[-3 for v in range(10)]
for x in nrc:
  if len(x)==3:
    vocab_nrc[x[0]][position[x[1]]]+=6*float(x[2])
print(nrc[:5])
print(vocab_nrc)

[[''], ['aback', 'anger', '0'], ['aback', 'anticipation', '0'], ['aback', 'disgust', '0'], ['aback', 'fear', '0']]
{'aback': [-3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0], 'abacus': [-3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, 3.0], 'abandon': [-3.0, -3.0, -3.0, 3.0, -3.0, 3.0, -3.0, 3.0, -3.0, -3.0], 'abandoned': [3.0, -3.0, -3.0, 3.0, -3.0, 3.0, -3.0, 3.0, -3.0, -3.0], 'abandonment': [3.0, -3.0, -3.0, 3.0, -3.0, 3.0, -3.0, 3.0, 3.0, -3.0], 'abate': [-3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0], 'abatement': [-3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0], 'abba': [-3.0, -3.0, -3.0, -3.0, -3.0, -3.0, 3.0, -3.0, -3.0, -3.0], 'abbot': [-3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, 3.0], 'abbreviate': [-3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0], 'abbreviation': [-3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0], 'abdomen': [-3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0], 'abdominal': [-3.

In [17]:
vocab_nrcs={}
positions={'anger':0,'anticip':1,'disgust':2,'fear':3,'joy':4,'negative':5,'positive':6,'sadness':7,'surprise':8,'trust':9}
with open("/content/drive/MyDrive/InfluenceAnalysis/InfluenceAnalysis/Data/NRC/NRC-Emotion-Lexicon-Senselevel-v0.92.txt", 'r', encoding="utf-8") as f:
  nrcs=[]
  for line in f:
      nrcs+=[line]
for x in range(len(nrcs)):
  nrcs[x]=nrcs[x].replace('--',' ')
  nrcs[x]=nrcs[x].replace(', ',' ')
  nrcs[x]=nrcs[x].replace('\t',' ')
  nrcs[x]=nrcs[x].replace('\n','')
  nrcs[x]=nrcs[x].split(' ')
for x in nrcs:
  for y in range(len(x)-2):
    vocab_nrcs[x[y]]=[-3 for v in range(10)]
for x in nrcs:
  for y in range(len(x)-2):  
    if float(x[-1])!=0:
      vocab_nrcs[x[y]][positions[x[-2]]]=3
print(nrcs[:5])
print(vocab_nrcs)

[['gut', 'opening', 'fistula', 'tubule', 'fear', '0'], ['gut', 'opening', 'fistula', 'tubule', 'anger', '0'], ['gut', 'opening', 'fistula', 'tubule', 'anticip', '0'], ['gut', 'opening', 'fistula', 'tubule', 'trust', '0'], ['gut', 'opening', 'fistula', 'tubule', 'surprise', '0']]
{'gut': [-3, -3, 3, -3, -3, -3, -3, -3, -3, -3], 'opening': [3, -3, 3, 3, -3, 3, 3, 3, 3, -3], 'fistula': [-3, -3, 3, -3, -3, -3, -3, -3, -3, -3], 'tubule': [-3, -3, 3, -3, -3, -3, -3, -3, -3, -3], 'retraction': [-3, -3, -3, -3, -3, 3, -3, -3, -3, -3], 'repentance': [-3, -3, -3, -3, -3, -3, -3, -3, -3, -3], 'withdrawal': [3, 3, -3, 3, -3, 3, 3, 3, -3, -3], 'apostasy': [-3, -3, -3, -3, -3, 3, -3, 3, -3, -3], 'conceit': [-3, 3, -3, -3, -3, 3, -3, -3, -3, -3], 'vanity': [-3, -3, 3, 3, -3, 3, 3, 3, -3, -3], 'assurance': [-3, -3, -3, -3, -3, 3, 3, -3, -3, 3], 'airs': [-3, -3, 3, -3, -3, 3, -3, -3, -3, -3], 'guide': [-3, -3, -3, -3, -3, -3, 3, -3, -3, 3], 'teaching': [3, 3, 3, 3, 3, 3, 3, -3, 3, 3], 'direct': [-3, -3

In [18]:
data_tweet = [nltk.pos_tag(x) for x in data_tweet]

In [19]:
und={}

for i in data_tweet:
  for j in i:
    und[j[1]]=1
len(und)

36

In [20]:
c=0
un={}
for i in data_tweet:
  for j in i:
    if j[1] not in un:
      un[j[1]]=[float(-3) for k in range(40)]
      un[j[1]][c]=float(3)
      c+=1
print(un)
print(len(un))

{'VBG': [3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0], 'IN': [-3.0, 3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0], 'NN': [-3.0, -3.0, 3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0], 'DT': [-3.0, -3.0, -3.0, 3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0], 'JJ': [-3.

In [21]:
def vec_gen(w2v,vocab,data_tweet,data_cat,batch_size,Max_input_size,Embedding_size):
  while True:
    for k in range(int(len(data_tweet)/batch_size)):
      temp = np.array([np.array([w2v.wv.get_vector(i) for i in x if i in vocab]) for x in data_tweet[k*(batch_size):(k+1)*(batch_size)]])
      temp = np.array([np.pad(x.flatten(),(0,Max_input_size*Embedding_size-len(x.flatten()))).reshape(Max_input_size,Embedding_size) for x in temp])
      tempres = np.array(data_cat[k*(batch_size):(k+1)*(batch_size)])
      yield (temp,tempres)

def glove_gen(vocab,data_tweet,data_cat,batch_size,Max_input_size,Embedding_size):
  while True:
    for k in range(int(len(data_tweet)/batch_size)):
      temp = np.array([np.array([vocab[i] for i in x if i in vocab.keys()]) for x in data_tweet[k*(batch_size):(k+1)*(batch_size)]])
      temp = np.array([np.pad(x.flatten(),(0,Max_input_size*Embedding_size-len(x.flatten()))).reshape(Max_input_size,Embedding_size) for x in temp])
      tempres = np.array(data_cat[k*(batch_size):(k+1)*(batch_size)])
      yield (temp,tempres)

def glove_gen_nrc(un,vocab_nrcs,vocab_nrc,vocab,data_tweet,data_cat,batch_size,Max_input_size,Embedding_size):
  while True:
    for k in range(int(len(data_tweet)/batch_size)):
      res=[]
      for x in data_tweet[k*(batch_size):(k+1)*(batch_size)]:
        tweet = [(np.array(vocab[i[0]]),i[0],i[1]) for i in x if i[0] in vocab.keys()]
        for b in range(len(tweet)):
          tweet[b]=(np.append(tweet[b][0],[vocab_nrc.get(tweet[b][1],[0 for aa in range(10)])+vocab_nrcs.get(tweet[b][1],[0 for aa in range(10)])+un.get(tweet[b][2],[0 for aa in range(40)])]),tweet[b][1])
        l=len(tweet)
        res+=[np.array([x[0] for x in tweet])]
      temp = np.array([np.pad(z.flatten(),(0,Max_input_size*(Embedding_size+60)-len(z.flatten()))).reshape(Max_input_size,(Embedding_size+60)) for z in res])
      tempres = np.array(data_cat[k*(batch_size):(k+1)*(batch_size)])
      yield (temp,tempres)

def vec_gen_nrc(un,vocab_nrcs,vocab_nrc,w2v,vocab,data_tweet,data_cat,batch_size,Max_input_size,Embedding_size):
  while True:
    for k in range(int(len(data_tweet)/batch_size)):
      res=[]
      for x in data_tweet[k*(batch_size):(k+1)*(batch_size)]:
        tweet = [(np.array(w2v.wv.get_vector(i[0])),i[0],i[1]) for i in x if i[0] in vocab]
        for b in range(len(tweet)):
          tweet[b]=(np.append(tweet[b][0],[vocab_nrc.get(tweet[b][1],[0 for aa in range(10)])+vocab_nrcs.get(tweet[b][1],[0 for aa in range(10)])+un.get(tweet[b][2],[0 for aa in range(40)])]),tweet[b][1])
        l=len(tweet)
        res+=[np.array([x[0] for x in tweet])]
      temp = np.array([np.pad(z.flatten(),(0,Max_input_size*(Embedding_size+60)-len(z.flatten()))).reshape(Max_input_size,(Embedding_size+60)) for z in res])
      tempres = np.array(data_cat[k*(batch_size):(k+1)*(batch_size)])
      yield (temp,tempres)


In [22]:
from sklearn.model_selection import StratifiedShuffleSplit
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=0)
sss.get_n_splits(data_tweet, data_cat)
X_train=[]
X_test=[]
y_train=[]
y_test=[]
counter=0
for train_index, test_index in sss.split(data_tweet, data_cat):
  print("TRAIN:", train_index, "TEST:", test_index)
  for i in train_index:
    X_train+=[data_tweet[i]]
    y_train+=[data_cat[i]]

  for j in test_index:
    X_test+=[data_tweet[j]]
    y_test+=[data_cat[j]]

 # X_train, X_test = data_tweet[train_index], data_tweet[test_index]
 # y_train, y_test = data_cat[train_index], data_cat[test_index]
print(X_train[:5])
print(y_train[:5])
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=0)
sss.get_n_splits(X_train, y_train)
x_train=[]
X_val=[]
Y_train=[]
y_val=[]
for train_index1, test_index1 in sss.split(X_train, y_train):
  print("TRAIN:", train_index1, "TEST:", test_index1)
  for i in train_index1:
    x_train+=[X_train[i]]
    Y_train+=[y_train[i]]

  for j in test_index1:
    X_val+=[X_train[j]]
    y_val+=[y_train[j]]
X_train=x_train
y_train=Y_train
print(len(X_train))
print(len(y_train))
print(len(X_test))
print(len(y_test))
print(len(X_val))
print(len(y_val))
print(X_train[:5])
print(y_train[:5])
result_table = [0,0,0,0,0,0,0,0,0]
  

TRAIN: [10470  6321 13813 ... 24250 39308  6334] TEST: [22946 37404 11668 ...  6796  7895 14585]
[[('the', 'DT'), ('goonies', 'NNS'), ('projected', 'VBN'), ('on', 'IN'), ('a', 'DT'), ('garage', 'NN'), ('door', 'NN'), ('with', 'IN'), ('my', 'PRP$'), ('friends', 'NNS'), ('amazing', 'VBG')], [('17', 'CD'), ('again', 'RB'), ('tonight', 'NN')], [('we', 'PRP'), ('finally', 'RB'), ('hit', 'VBD'), ('1000', 'CD'), ('friends', 'NNS'), ('thanks', 'NNS')], [('lolzz', 'JJ'), ('public', 'JJ'), ('script', 'NN'), ('wait', 'NN'), ('n', 'NN'), ('watch', 'NN')], [('night', 'NN')]]
[array([0, 1, 0, 0]), array([1, 0, 0, 0]), array([0, 1, 0, 0]), array([0, 1, 0, 0]), array([1, 0, 0, 0])]
TRAIN: [28318 14444 15946 ... 11043 21820 15291] TEST: [23742   966  6886 ... 25081 24593 26654]
25600
25600
8000
8000
6400
6400
[[('geez', 'JJ'), ('15', 'CD'), ('years', 'NNS'), ('you', 'PRP'), ('d', 'VBP'), ('think', 'VBP'), ('they', 'PRP'), ('would', 'MD'), ('have', 'VB'), ('had', 'VBN'), ('that', 'IN'), ('checked', 'VBN

In [23]:
epochs = 40

In [24]:
from keras import backend as K
def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recalls = true_positives / (possible_positives + K.epsilon())
        return recalls
def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precisions = true_positives / (predicted_positives + K.epsilon())
        return precisions
def f1(y_true, y_pred):
    precisions = precision(y_true, y_pred)
    recalls = recall(y_true, y_pred)
    return 2*((precisions*recalls)/(precisions+recalls+K.epsilon()))

In [25]:
sg_train_gen=vec_gen_nrc(un,vocab_nrcs,vocab_nrc,w2v_sg,vocab_sg,X_train,y_train,batch_size,Max_input_size,Embedding_size)
sg_val_gen=vec_gen_nrc(un,vocab_nrcs,vocab_nrc,w2v_sg,vocab_sg,X_val,y_val,batch_size,Max_input_size,Embedding_size)
cbow_train_gen=vec_gen_nrc(un,vocab_nrcs,vocab_nrc,w2v_cbow,vocab_cbow,X_train,y_train,batch_size,Max_input_size,Embedding_size)
cbow_val_gen=vec_gen_nrc(un,vocab_nrcs,vocab_nrc,w2v_cbow,vocab_cbow,X_val,y_val,batch_size,Max_input_size,Embedding_size)

In [26]:
model_sg_bi = tf.keras.Sequential()
model_sg_bi.add(tf.keras.Input(shape=(Max_input_size,Embedding_size+60)))
model_sg_bi.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64),merge_mode='concat'))
model_sg_bi.add(tf.keras.layers.Dropout(0.2))
model_sg_bi.add(tf.keras.layers.Dense(64,activation='relu'))
model_sg_bi.add(tf.keras.layers.Dropout(0.2))
model_sg_bi.add(tf.keras.layers.Dense(4, activation='softmax'))
model_sg_bi.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy',f1,precision,recall])
history_sg_bi = model_sg_bi.fit_generator(sg_train_gen,validation_data=sg_val_gen,validation_steps=int((len(X_val)/batch_size)+1),steps_per_epoch=int((len(X_train)/batch_size)+1),epochs=epochs)
history = model_sg_bi.evaluate(x = vec_gen_nrc(un,vocab_nrcs,vocab_nrc,w2v_sg,vocab_sg,X_test,y_test,batch_size,Max_input_size,Embedding_size),steps=int(len(y_test)/batch_size))
print(history)

result_table[1]=history[1]
print("SG_BI")

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/40
26/26 [==============================] - 92s 3s/step - loss: 1.3754 - accuracy: 0.2854 - f1: 0.0327 - precision: 0.3890 - recall: 0.0177 - val_loss: 1.1748 - val_accuracy: 0.4341 - val_f1: 0.0489 - val_precision: 0.7331 - val_recall: 0.0253
Epoch 2/40
26/26 [==============================] - 86s 3s/step - loss: 1.1943 - accuracy: 0.4002 - f1: 0.0899 - precision: 0.5425 - recall: 0.0494 - val_loss: 1.1449 - val_accuracy: 0.4687 - val_f1: 0.1858 - val_precision: 0.6259 - val_recall: 0.1093
Epoch 3/40
26/26 [==============================] - 87s 3s/step - loss: 1.1541 - accuracy: 0.4610 - f1: 0.2382 - precision: 0.5639 - recall: 0.1521 - val_loss: 1.1171 - val_accuracy: 0.4986 - val_f1: 0.2820 - val_precision: 0.6182 - val_recall: 0.1827
Epoch 4/40
26/26 [==============================] - 88s 3s/step - loss: 1.1280 - accuracy: 0.4862 - f1: 0.2967 - precision: 0.5955 - recall: 0.1991 - val_loss: 1.1022 - val_accuracy: 0.5080 - val_f1: 0.3259 - val_precision: 0.6208 - val_recall:

In [27]:
model_cbow_bi = tf.keras.Sequential()
model_cbow_bi.add(tf.keras.Input(shape=(Max_input_size,Embedding_size+60)))
model_cbow_bi.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64),merge_mode='concat'))
model_cbow_bi.add(tf.keras.layers.Dropout(0.2))
model_cbow_bi.add(tf.keras.layers.Dense(64,activation='relu'))
model_cbow_bi.add(tf.keras.layers.Dropout(0.2))
model_cbow_bi.add(tf.keras.layers.Dense(4, activation='softmax'))
model_cbow_bi.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy',f1,precision,recall])
history_cbow_bi = model_cbow_bi.fit_generator(cbow_train_gen,validation_data=cbow_val_gen,validation_steps=int((len(X_val)/batch_size)+1),steps_per_epoch=int((len(X_train)/batch_size)+1),epochs=epochs)
history = model_cbow_bi.evaluate(x = vec_gen_nrc(un,vocab_nrcs,vocab_nrc,w2v_cbow,vocab_cbow,X_test,y_test,batch_size,Max_input_size,Embedding_size),steps=int(len(y_test)/batch_size))
print(history)

result_table[4]=history[1]
print("CBOW_BI")

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/40
26/26 [==============================] - 98s 4s/step - loss: 1.2384 - accuracy: 0.3644 - f1: 0.0190 - precision: 0.2798 - recall: 0.0099 - val_loss: 1.1687 - val_accuracy: 0.4503 - val_f1: 0.0544 - val_precision: 0.7243 - val_recall: 0.0283
Epoch 2/40
26/26 [==============================] - 94s 4s/step - loss: 1.1755 - accuracy: 0.4299 - f1: 0.1092 - precision: 0.6113 - recall: 0.0615 - val_loss: 1.1310 - val_accuracy: 0.4771 - val_f1: 0.2609 - val_precision: 0.6043 - val_recall: 0.1664
Epoch 3/40
26/26 [==============================] - 92s 4s/step - loss: 1.1401 - accuracy: 0.4768 - f1: 0.2700 - precision: 0.5862 - recall: 0.1758 - val_loss: 1.1065 - val_accuracy: 0.4984 - val_f1: 0.2927 - val_precision: 0.6319 - val_recall: 0.1904
Epoch 4/40
26/26 [==============================] - 93s 4s/step - loss: 1.1168 - accuracy: 0.4957 - f1: 0.3291 - precision: 0.6082 - recall: 0.2261 - val_loss: 1.0944 - val_accuracy: 0.5094 - val_f1: 0.3360 - val_precision: 0.6283 - val_recall:

In [28]:
model_glove_bi = tf.keras.Sequential()
model_glove_bi.add(tf.keras.Input(shape=(Max_input_size,Embedding_size+60)))
model_glove_bi.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64),merge_mode='concat'))
model_glove_bi.add(tf.keras.layers.Dropout(0.2))
model_glove_bi.add(tf.keras.layers.Dense(64,activation='relu'))
model_glove_bi.add(tf.keras.layers.Dropout(0.2))
model_glove_bi.add(tf.keras.layers.Dense(4, activation='softmax'))
model_glove_bi.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy',f1,precision,recall])
glove_train_gen=glove_gen_nrc(un,vocab_nrcs,vocab_nrc,vocab_glove,X_train,y_train,batch_size,Max_input_size,Embedding_size)
glove_val_gen=glove_gen_nrc(un,vocab_nrcs,vocab_nrc,vocab_glove,X_val,y_val,batch_size,Max_input_size,Embedding_size)
history_glove_bi = model_glove_bi.fit_generator(glove_train_gen,validation_data=glove_val_gen,validation_steps=int((len(X_val)/batch_size)+1),steps_per_epoch=int((len(X_train)/batch_size)+1),epochs=epochs)
history = model_glove_bi.evaluate(x = glove_gen_nrc(un,vocab_nrcs,vocab_nrc,vocab_glove,X_test,y_test,batch_size,Max_input_size,Embedding_size),steps=int(len(y_test)/batch_size))
print(history)

result_table[7]=history[1]
print("GLOVE_BI")

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/40
26/26 [==============================] - 47s 2s/step - loss: 1.2437 - accuracy: 0.3811 - f1: 0.0520 - precision: 0.4663 - recall: 0.0279 - val_loss: 1.1421 - val_accuracy: 0.4659 - val_f1: 0.1380 - val_precision: 0.6668 - val_recall: 0.0771
Epoch 2/40
26/26 [==============================] - 42s 2s/step - loss: 1.1384 - accuracy: 0.4734 - f1: 0.2521 - precision: 0.5967 - recall: 0.1634 - val_loss: 1.0733 - val_accuracy: 0.5273 - val_f1: 0.3972 - val_precision: 0.6254 - val_recall: 0.2910
Epoch 3/40
26/26 [==============================] - 41s 2s/step - loss: 1.0661 - accuracy: 0.5316 - f1: 0.4093 - precision: 0.6340 - recall: 0.3033 - val_loss: 1.0412 - val_accuracy: 0.5481 - val_f1: 0.4289 - val_precision: 0.6433 - val_recall: 0.3217
Epoch 4/40
26/26 [==============================] - 41s 2s/step - loss: 1.0365 - accuracy: 0.5547 - f1: 0.4534 - precision: 0.6568 - recall: 0.3478 - val_loss: 1.0424 - val_accuracy: 0.5383 - val_f1: 0.4499 - val_precision: 0.6445 - val_recall:

In [29]:
print("\tLstm\t\t\tBiLstm\t\t\tRnn")
print("sg",result_table[:3])
print("cbow",result_table[3:6])
print("glove",result_table[6:9])

	Lstm			BiLstm			Rnn
sg [0, 0.543749988079071, 0]
cbow [0, 0.5270000100135803, 0]
glove [0, 0.5576249957084656, 0]
